In [1]:
#Pretty Display of Variables
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#%autoreload

# Import libraries

In [3]:
from plot_projections import plot_all_projections, plot_final_projections, plot_point_for_couple, plot_projection_up_down, plot_border
from prop_info import extreme_points, vect_blade, d_blade, center_prop
from get_segments import blade_alone, get_segments_points, get_planes
from major_axis import get_major_axis
#from projections import couple_all_planes, project_all_couples, projections_by_side, project_couple
from new_projections import assign_points, get_all_points_for_projections, interpolations, find_separation_plane, interpolate_points, model_func
from parameters import get_hub_points, get_hub_radius
from plot_param import plot_hub
from plot_prop import plot_pointcloud, plot_direction, plot_segments
from myMathFunction import least_squares
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# Pre-processing

In [4]:
propeller_coords = pd.read_csv('aerostar_data.csv')
propeller_coords.shape

(151986, 3)

In [5]:
propeller_coords = propeller_coords.drop_duplicates(subset=None, keep='first', inplace=False)
propeller_coords = propeller_coords.reset_index(drop=True)

In [6]:
max_point, min_point, middle_point, highest_point, lowest_point = extreme_points(propeller_coords)

vect_length = vect_blade(max_point, min_point) 

dmiddle, dhighest, dlowest = d_blade(vect_length, middle_point, highest_point, lowest_point)

upper_blade, lower_blade = blade_alone(propeller_coords, vect_length, dmiddle)

vect_out, vect_side, hub_inner_radius = get_major_axis(propeller_coords, middle_point, vect_length)

# Projections

In [7]:
nb_seg = 3
resolution = 3

planes = get_planes(upper_blade, dmiddle, dhighest, vect_length, nb_seg)
segments = get_segments_points(upper_blade, planes, nb_seg)
#plot_segments(segments)

up, down = get_all_points_for_projections(planes, segments, nb_seg, resolution)

In [8]:
up1 = up[0]
dn1 = down[0]
print(up1.shape)

(322, 3)


In [9]:
# 1. Find border points
up_right_border, up_left_border, _, _, _ = extreme_points(up1)
dn_right_border, dn_left_border, _, _, _ = extreme_points(dn1)
#print("up_right {}\n".format(up_right))

In [10]:
# 2. Find separating plane
C_up = find_separation_plane(up1.values)
C_dn = find_separation_plane(dn1.values)
#print("C_up {}\n".format(C_up))

In [11]:
# 3. Assign point to side  (do it for both sides on both sides)
up_right_points, up_left_points = assign_points(C_up, up1)
dn_right_points, dn_left_points = assign_points(C_dn, dn1)
#print("right_points_up_shape {}\n".format(right_points_up.shape))

In [ ]:
################################# TO DO ####################################
# Add border points to fit
up_right_points.append(pd.DataFrame(up_right_border))
up_right_points.append(pd.DataFrame(up_left_border))
#print("right_points_up_shape {}\n".format(right_points_up.shape))

In [14]:
# 4. Interpolate points
up_right_popt = interpolate_points(up_right_points)
up_left_popt = interpolate_points(up_left_points)
dn_right_popt = interpolate_points(dn_right_points)
dn_left_popt = interpolate_points(dn_left_points)

In [23]:
# 5. Final projection
nb_points = 100
# Prendre 100 points d'un côté à l'autre de chaque courbe
range_X_up = np.linspace(up_right_border[0], up_left_border[0], nb_points)
range_Y_up = np.linspace(up_right_border[1], up_left_border[1], nb_points)

range_X_dn = np.linspace(dn_right_border[0], dn_left_border[0], nb_points)
range_Y_dn = np.linspace(dn_right_border[1], dn_left_border[1], nb_points)

# Relier les points d'en haut avec ceux du bas



# Garder les projections sur le plan



In [26]:
range_X_up - range_X_dn

array([ 3.23412895,  3.23143227,  3.22873559,  3.2260389 ,  3.22334222,
        3.22064554,  3.21794886,  3.21525217,  3.21255549,  3.20985881,
        3.20716212,  3.20446544,  3.20176876,  3.19907208,  3.19637539,
        3.19367871,  3.19098203,  3.18828535,  3.18558866,  3.18289198,
        3.1801953 ,  3.17749862,  3.17480193,  3.17210525,  3.16940857,
        3.16671188,  3.1640152 ,  3.16131852,  3.15862184,  3.15592515,
        3.15322847,  3.15053179,  3.14783511,  3.14513842,  3.14244174,
        3.13974506,  3.13704837,  3.13435169,  3.13165501,  3.12895833,
        3.12626164,  3.12356496,  3.12086828,  3.1181716 ,  3.11547491,
        3.11277823,  3.11008155,  3.10738486,  3.10468818,  3.1019915 ,
        3.09929482,  3.09659813,  3.09390145,  3.09120477,  3.08850809,
        3.0858114 ,  3.08311472,  3.08041804,  3.07772135,  3.07502467,
        3.07232799,  3.06963131,  3.06693462,  3.06423794,  3.06154126,
        3.05884458,  3.05614789,  3.05345121,  3.05075453,  3.04

# Find parameters

    - Tip radius 
        Projection dans le sens de la longueur et ... ??
    - Blade twist 
        Project on planes as in projection
        Compute difference of angle between main directions of projections (PCA possible)

### Hub radius

In [ ]:
hub_points = get_hub_points(propeller_coords, dmiddle, vect_length)

hub_outer_radius, hub_inner_radius = get_hub_radius(hub_points, middle_point, vect_side)

hub_radius = hub_outer_radius[2] - middle_point[2]  #from center to exterior radius

In [ ]:
plot_hub(propeller_coords, hub_points, hub_outer_radius, hub_inner_radius)

### Chord length

In [ ]:
chord_length = np.linalg.norm(highest_point - middle_point) - hub_radius
chord_length

### Blade twist

### Tip radius